In [1]:
from docx import Document
import os
import re
import json
import pandas as pd

In [2]:
def hebrew_to_number(hebrew_text):
    hebrew_numerals = {
        "אחד": 1, "שתיים": 2, "שלוש": 3, "ארבע": 4, "חמש": 5,
        "שש": 6, "שבע": 7, "שמונה": 8, "תשע": 9, "עשר": 10,
        "עשרים": 20, "שלושים": 30, "ארבעים": 40, "חמישים": 50,
        "שישים": 60, "שבעים": 70, "שמונים": 80, "תשעים": 90,
        "מאה": 100, "מאתיים": 200
    }

    suffixes = {
        "מאות": 100, "עשרה" : 10
    }

    total = 0
    parts = re.split(r'-', hebrew_text)
    # by now we should have an array of the hebrew numbers/suffixes

    for i, part in enumerate(parts):
        part = part.strip()
        
        if part.startswith('ו'):
            part = part[1:]

        if part in hebrew_numerals:
            if i + 1 < len(parts) and parts[i+1] in suffixes:
                if parts[i+1] == "מאות":
                    total += hebrew_numerals[part] * suffixes[parts[i+1]]
                elif parts[i+1] == "עשרה":
                    total += hebrew_numerals[part] + suffixes[parts[i+1]]
                continue
            else:
                total += hebrew_numerals[part]
                
    return total

In [3]:
def parse_protocol_metadata(file_path):
    filename = file_path.split('/')[-1]

    ## knesset num ##
    knesset_num = filename.split('_')[0]
    knesset_num = knesset_num.split('\\')[-1]

    ## type ##
    if filename.split('_')[1][2] == 'm':
        protocol_type = 'plenary'
    elif filename.split('_')[1][2] == 'v':
        protocol_type = 'committee'

    ## protocol number ##
    protocol_number = -1

    document = Document(file_path)
    lines = [para.text for para in document.paragraphs]

    for line in lines[:500]:
        if "פרוטוקול מס'" in line:
            match = re.search(r"פרוטוקול מס'\s*(\d+)", line)
            if match:
                protocol_number = match.group(1)
        if re.search(r"הישיבה ה.*? של הכנסת ה.*", line):
            match = re.search(r"הישיבה ה(.*?) של", line)           # get the hebrew number only
            hebrew_number = match.group(1).strip()                  # extract the text
            protocol_number = hebrew_to_number(hebrew_number)       # use a function to determine the number
    
    return knesset_num, protocol_type, protocol_number

In [4]:
def clean_speaker_name(raw_name):
    prefixes_to_remove = [
        r'היו"ר', r'יו"ר', r'ח"כ', r'ד"ר', r'פרופ\'', r'עו"ד', 
        r'נצ"מ', r'ניצב',
        r'מר', r'גברת',
        r'שר', r'שרת', r'השר', r'השרה',
        r'סגן', r'סגנית',
        r'הבינוי והשיכון',
        r'העבודה הרווחה והשירותים החברתיים',
        r'האוצר',
        r'התעשייה והמסחר',
        r'החינוך',
        r'מזכיר המדינה',
        r'מזכיר הכנסת',
        r'התחבורה',
        r'הפנים',
        r'המשפטים',
        r'והתרבות',
        r'הכלכלה והתכנון',
        r'לביטחון פנים',
        r'לביטחון'
    ]

    # Create a regex pattern to match any of the prefixes
    prefix_pattern = r'\b(?:' + r'|'.join(prefixes_to_remove) + r')\b'

    cleaned_name = re.sub(r'\(.*?\)', '', raw_name)
    cleaned_name = re.sub(prefix_pattern, '', cleaned_name)
    cleaned_name = re.sub(r'[<>]', '', cleaned_name)
    return cleaned_name.strip()

In [5]:
def extract_sentences(file_path):
    data = []
    found_first_speaker = False

    doc = Document(file_path)
    speaker = None
    speech = []

    for para in doc.paragraphs:
        if(not found_first_speaker):    # discussion has not begun
            # check if the current line is underlined
            try:
                is_underlined = (
                    para.style.font.underline or 
                    (para.style.base_style and para.style.base_style.font.underline) or 
                    (para.runs and para.runs[0].underline)
                )
            except AttributeError:
                is_underlined = False
            
            # if the current line is underlined, has "היו"ר" and a colon, 
            if(is_underlined and 
               (re.search(r'היו"ר .*:', para.text) or
                re.search(r'היו”ר .*:', para.text) or
                re.search(r'יו"ר .*:', para.text) or
                re.search(r'יו”ר .*:', para.text) or
                # This case occurs once in the files so we used this specific case
                ('16_ptv_577758' in file_path and re.search(r'שלמי גולדברג .*:', para.text)))
               ):
                match = re.match(r'(.*?):\s*(.*)', para.text)
                if match:
                    speaker = match.group(1).strip()
                    speaker = clean_speaker_name(speaker)
                    speech = [match.group(2).strip().replace(">","")]
                    if speech != ['']:
                        data.append({"Speaker": speaker, "Speech": " ".join(speech)})
                found_first_speaker = True
                continue
        else:                           # discussion has begun
            if(para.alignment == 1):    # 1:centered  
                continue                # skip centered paragraphs (titles, votes etc...)
            
            # check if the current line is underlined
            try:
                is_underlined = (
                    para.style.font.underline or 
                    (para.style.base_style and para.style.base_style.font.underline) or 
                    (para.runs and para.runs[0].underline)
                )
            except AttributeError:
                is_underlined = False

            if ":" in para.text and is_underlined:      # potential speaker
                match = re.match(r'(.*?):\s*(.*)', para.text)
                if match:
                    if speaker and speech != ['']:
                        data.append({"Speaker": speaker, "Speech": " ".join(speech)})
                        speech = []
                    
                    # update the speaker and speech
                    speaker = match.group(1).strip()
                    speaker = clean_speaker_name(speaker)
                    speech.append(match.group(2).strip().replace(">",""))
            elif speaker and speech:
                if "הישיבה ננעלה בשעה" in speech or "הישיבה ננעלה בשעה" in speaker:
                    continue
                text = para.text.strip()
                if text:
                    speech.append(text.replace(">",""))
    
    if speaker:
        data.append({"Speaker": speaker, "Speech": " ".join(speech)})
    
    return data


In [6]:
def produce_corpus(dir_path):
    data = []
    
    for curr_file in os.listdir(dir_path):
        file_path = os.path.join(dir_path, os.fsdecode(curr_file))
        knesset_num, protocol_type, protocol_num = parse_protocol_metadata(file_path)
        data = extract_sentences(file_path)

        # write to file
        output_file = os.path.splitext(curr_file)[0] + ".txt"
        output_path = os.path.join("output", output_file)

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(f"Knesset Number: {knesset_num}\n")
            f.write(f"Type: {protocol_type}\n")
            f.write(f"Protocol Number: {protocol_num}\n\n")
            for entry in data:
                f.write(f"Speaker: {entry['Speaker']}\n")
                f.write(f"Speech: {entry['Speech']}\n")
                f.write("\n")

In [7]:
dir = 'protocols'
produce_corpus(dir)